In [1]:
import pandas as pd
from pandas import ExcelFile
from pandas.plotting import scatter_matrix
import numpy as np
from datetime import datetime
import time
import requests
import json

import matplotlib.pyplot as plt

import nba_api
from nba_api.stats.static import teams
from nba_api.stats.endpoints import LeagueGameFinder, LeagueGameLog, PlayerGameLog
from nba_api.stats.endpoints import BoxScoreDefensive, BoxScoreMiscV2, BoxScorePlayerTrackV2, BoxScoreUsageV2

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, Normalizer, StandardScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

-------
# csv dataset
-------

In [4]:
df = pd.read_csv('gamedata_test_reduced.csv')

In [5]:
df

,SEASON_ID,PLAYER_ID,GAME_ID,GAME_DATE_x,MATCHUP,WL,MIN_x,FGM,FGA,FG_PCT_x,...,PCT_DREB,PCT_REB,PCT_AST,PCT_TOV,PCT_STL,PCT_BLK,PCT_BLKA,PCT_PF,PCT_PFD,PCT_PTS
0,22018,203500,21801197,7-Apr-19,OKC @ MIN,W,27,4,5,0.800,...,0.174,0.207,0.056,0.300,0.200,0.000,0.000,0.353,0.167,0.092
1,22018,203500,21801186,5-Apr-19,OKC vs. DET,W,24,7,9,0.778,...,0.273,0.424,0.000,0.222,0.000,1.000,0.000,0.364,0.000,0.226
2,22018,203500,21801161,2-Apr-19,OKC vs. LAL,W,33,6,11,0.545,...,0.042,0.205,0.111,0.333,0.091,0.833,0.500,0.250,0.389,0.141
3,22018,203500,21801146,31-Mar-19,OKC vs. DAL,L,39,10,17,0.588,...,0.214,0.349,0.111,0.000,0.000,0.400,0.000,0.111,0.200,0.233
4,22018,203500,21801133,29-Mar-19,OKC vs. DEN,L,35,8,12,0.667,...,0.158,0.200,0.125,0.111,0.000,0.333,0.000,0.000,0.063,0.186
5,22018,203500,21801119,27-Mar-19,OKC vs. IND,W,35,11,14,0.786,...,0.192,0.343,0.136,0.000,0.286,0.200,0.200,0.111,0.214,0.298
6,22018,203500,21801104,25-Mar-19,OKC @ MEM,L,32,1,7,0.143,...,0.111,0.296,0.000,0.167,0.800,1.000,0.000,0.154,0.000,0.030
7,22018,203500,21801083,22-Mar-19,OKC @ TOR,W,33,4,9,0.444,...,0.053,0.167,0.045,0.111,0.250,0.600,0.000,0.143,0.158,0.110
8,22018,203500,21801072,20-Mar-19,OKC vs. TOR,L,40,2,7,0.286,...,0.000,0.171,0.133,0.133,0.250,0.000,0.000,0.200,0.118,0.045
9,22018,203500,21801054,18-Mar-19,OKC vs. MIA,L,25,5,6,0.833,...,0.381,0.414,0.231,0.182,0.500,0.400,0.000,0.545,0.154,0.169


In [12]:
list(df.keys())

['SEASON_ID',
 'PLAYER_ID',
 'GAME_ID',
 'GAME_DATE_x',
 'MATCHUP',
 'WL',
 'MIN_x',
 'FGM',
 'FGA',
 'FG_PCT_x',
 'FG3M',
 'FG3A',
 'FG3_PCT',
 'FTM',
 'FTA',
 'FT_PCT',
 'OREB',
 'DREB_x',
 'REB',
 'AST_x',
 'STL_x',
 'BLK_x',
 'TOV',
 'PF_x',
 'PTS',
 'PLUS_MINUS',
 'VIDEO_AVAILABLE',
 'TEAM_ID',
 'TEAM_ABBREVIATION_x',
 'TEAM_CITY_x',
 'PLAYER_NAME_x',
 'START_POSITION_x',
 'COMMENT_x',
 'MIN_y',
 'E_OFF_RATING',
 'OFF_RATING',
 'E_DEF_RATING',
 'DEF_RATING',
 'E_NET_RATING',
 'NET_RATING',
 'AST_PCT',
 'AST_TOV',
 'AST_RATIO',
 'OREB_PCT',
 'DREB_PCT',
 'REB_PCT',
 'TM_TOV_PCT',
 'EFG_PCT',
 'TS_PCT',
 'USG_PCT_x',
 'E_USG_PCT',
 'E_PACE',
 'PACE',
 'PACE_PER40',
 'POSS',
 'PIE',
 'Gamescore',
 'FT_missed',
 'GAME_DATE_y',
 'attendance',
 'arena',
 'City',
 'Hometeam',
 'Capacity',
 'Year Built',
 'Home/Away',
 'Capacity_pct',
 'TEAM_ABBREVIATION_y',
 'TEAM_CITY_y',
 'TEAM_NICKNAME',
 'PLAYER_NAME_y',
 'START_POSITION_y',
 'COMMENT_y',
 'MATCHUP_MIN',
 'PARTIAL_POSS',
 'SWITCHES_O

In [10]:
performance_categories = [
    'PCT_FGM',
     'PCT_FGA',
     'PCT_FG3M',
     'PCT_FG3A',
     'PCT_FTM',
     'PCT_FTA',
     'PCT_OREB',
     'PCT_DREB',
     'PCT_REB',
     'PCT_AST',
     'PCT_TOV',
     'PCT_STL',
     'PCT_BLK',
     'PCT_BLKA',
     'PCT_PF',
     'PCT_PFD',
     'PCT_PTS'
]

In [14]:
tp_list = []

for index, row in df.iterrows():
    
    true_performance = (
        row['PCT_FGM'] +
         row['PCT_FGA'] +
         row['PCT_FG3M'] +
         row['PCT_FG3A'] +
         row['PCT_FTM'] +
         row['PCT_FTA'] +
         row['PCT_OREB'] +
         row['PCT_DREB'] +
         row['PCT_REB'] +
         row['PCT_AST'] +
         row['PCT_TOV'] +
         row['PCT_STL'] +
         row['PCT_BLK'] +
         row['PCT_BLKA'] +
         row['PCT_PF'] +
         row['PCT_PFD'] +
         row['PCT_PTS']
    ) / len(performance_categories)
    
    tp_list.append(true_performance)
    
    print(row['GAME_ID'])
    print(row['PLAYER_NAME'])
    print(true_performance)
    print('----------')

21801197
Steven Adams
0.12335294117647058
----------
21801186
Steven Adams
0.21617647058823528
----------
21801161
Steven Adams
0.24358823529411766
----------
21801146
Steven Adams
0.17276470588235296
----------
21801133
Steven Adams
0.11770588235294117
----------
21801119
Steven Adams
0.2354117647058824
----------
21801104
Steven Adams
0.1981764705882353
----------
21801083
Steven Adams
0.1507647058823529
----------
21801072
Steven Adams
0.0981764705882353
----------
21801054
Steven Adams
0.23535294117647054
----------
21801038
Steven Adams
0.1926470588235294
----------
21801020
Steven Adams
0.2291176470588235
----------
21801017
Steven Adams
0.17647058823529413
----------
21801005
Steven Adams
0.1936470588235294
----------
21800985
Steven Adams
0.13847058823529415
----------
21800976
Steven Adams
0.1440588235294118
----------
21800963
Steven Adams
0.2390588235294117
----------
21800951
Steven Adams
0.2287058823529412
----------
21800940
Steven Adams
0.3534705882352942
----------
2180

21800234
Kyle Anderson
0.112
----------
21800220
Kyle Anderson
0.11458823529411764
----------
21800207
Kyle Anderson
0.17194117647058824
----------
21800194
Kyle Anderson
0.17258823529411765
----------
21800179
Kyle Anderson
0.17129411764705885
----------
21800158
Kyle Anderson
0.1743529411764706
----------
21800147
Kyle Anderson
0.14123529411764704
----------
21800136
Kyle Anderson
0.16799999999999998
----------
21800122
Kyle Anderson
0.12788235294117647
----------
21800102
Kyle Anderson
0.17647058823529416
----------
21800080
Kyle Anderson
0.14952941176470588
----------
21800061
Kyle Anderson
0.15058823529411763
----------
21800045
Kyle Anderson
0.1875882352941176
----------
21800020
Kyle Anderson
0.22329411764705884
----------
21800005
Kyle Anderson
0.12888235294117645
----------
21801212
Ryan Anderson
0.0
----------
21801153
Ryan Anderson
0.12241176470588234
----------
21801141
Ryan Anderson
0.11276470588235293
----------
21801085
Ryan Anderson
0.09664705882352942
----------
218010

21800525
Deandre Ayton
0.14
----------
21800503
Deandre Ayton
0.16558823529411767
----------
21800489
Deandre Ayton
0.2962352941176471
----------
21800480
Deandre Ayton
0.26111764705882357
----------
21800458
Deandre Ayton
0.20417647058823526
----------
21800444
Deandre Ayton
0.25170588235294117
----------
21800435
Deandre Ayton
0.25364705882352945
----------
21800419
Deandre Ayton
0.20111764705882357
----------
21800403
Deandre Ayton
0.20711764705882352
----------
21800399
Deandre Ayton
0.3310588235294118
----------
21800376
Deandre Ayton
0.26399999999999996
----------
21800366
Deandre Ayton
0.26123529411764707
----------
21800353
Deandre Ayton
0.1912941176470588
----------
21800337
Deandre Ayton
0.24541176470588233
----------
21800327
Deandre Ayton
0.22929411764705884
----------
21800315
Deandre Ayton
0.2165294117647059
----------
21800305
Deandre Ayton
0.25635294117647056
----------
21800287
Deandre Ayton
0.3075882352941176
----------
21800275
Deandre Ayton
0.17635294117647057
-----

21800756
Nicolas Batum
0.21982352941176472
----------
21800744
Nicolas Batum
0.17129411764705882
----------
21800727
Nicolas Batum
0.14129411764705882
----------
21800713
Nicolas Batum
0.189
----------
21800688
Nicolas Batum
0.15505882352941178
----------
21800679
Nicolas Batum
0.17076470588235293
----------
21800666
Nicolas Batum
0.20635294117647057
----------
21800647
Nicolas Batum
0.11464705882352941
----------
21800636
Nicolas Batum
0.23905882352941177
----------
21800628
Nicolas Batum
0.15058823529411766
----------
21800606
Nicolas Batum
0.15699999999999997
----------
21800590
Nicolas Batum
0.11805882352941174
----------
21800576
Nicolas Batum
0.15623529411764703
----------
21800554
Nicolas Batum
0.15888235294117645
----------
21800543
Nicolas Batum
0.1048235294117647
----------
21800530
Nicolas Batum
0.16658823529411765
----------
21800517
Nicolas Batum
0.15776470588235295
----------
21800505
Nicolas Batum
0.10882352941176472
----------
21800488
Nicolas Batum
0.1198235294117647
-

0.03964705882352941
----------
21800232
Jordan Bell
0.17647058823529413
----------
21800213
Jordan Bell
0.1502941176470588
----------
21800201
Jordan Bell
0.27835294117647064
----------
21800198
Jordan Bell
0.1823529411764706
----------
21800182
Jordan Bell
0.14441176470588235
----------
21800166
Jordan Bell
0.1332941176470588
----------
21800147
Jordan Bell
0.09376470588235293
----------
21800124
Jordan Bell
0.03264705882352941
----------
21800108
Jordan Bell
0.21035294117647058
----------
21800091
Jordan Bell
0.1866470588235294
----------
21800083
Jordan Bell
0.09399999999999999
----------
21800068
Jordan Bell
0.14011764705882354
----------
21800062
Jordan Bell
0.11452941176470588
----------
21800047
Jordan Bell
0.14976470588235297
----------
21800038
Jordan Bell
0.09605882352941177
----------
21800002
Jordan Bell
0.13823529411764707
----------
21801220
DeAndre' Bembry
0.14435294117647063
----------
21801202
DeAndre' Bembry
0.30882352941176466
----------
21801181
DeAndre' Bembry
0.24

...

0.1313529411764706
----------
21800882
Gary Trent Jr.
0.02311764705882353
----------
21800816
Gary Trent Jr.
0.029411764705882353
----------
21800677
Gary Trent Jr.
0.0
----------
21800662
Gary Trent Jr.
0.0
----------
21800628
Gary Trent Jr.
0.176
----------
21800615
Gary Trent Jr.
0.18823529411764706
----------
21800366
Gary Trent Jr.
0.1673529411764706
----------
21800221
Gary Trent Jr.
0.16511764705882356
----------
21800138
Gary Trent Jr.
0.10429411764705881
----------
21801053
Allonzo Trier
0.21452941176470586
----------
21801042
Allonzo Trier
0.17476470588235293
----------
21801032
Allonzo Trier
0.18523529411764705
----------
21801007
Allonzo Trier
0.15323529411764708
----------
21800998
Allonzo Trier
0.27288235294117646
----------
21800986
Allonzo Trier
0.18788235294117647
----------
21800972
Allonzo Trier
0.1758823529411765
----------
21800957
Allonzo Trier
0.3489999999999999
----------
21800947
Allonzo Trier
0.1901764705882353
----------
21800924
Allonzo Trier
0.1851764705882

21800054
Noah Vonleh
0.17776470588235296
----------
21800042
Noah Vonleh
0.35152941176470587
----------
21800028
Noah Vonleh
0.21394117647058827
----------
21800018
Noah Vonleh
0.30770588235294116
----------
21800007
Noah Vonleh
0.2211176470588235
----------
21801203
Nikola Vucevic
0.25794117647058823
----------
21801181
Nikola Vucevic
0.23623529411764707
----------
21801166
Nikola Vucevic
0.2944117647058824
----------
21801156
Nikola Vucevic
0.263235294117647
----------
21801140
Nikola Vucevic
0.25111764705882356
----------
21801122
Nikola Vucevic
0.19900000000000004
----------
21801109
Nikola Vucevic
0.254
----------
21801103
Nikola Vucevic
0.2982941176470589
----------
21801080
Nikola Vucevic
0.24152941176470588
----------
21801066
Nikola Vucevic
0.35205882352941176
----------
21801046
Nikola Vucevic
0.3142352941176471
----------
21801021
Nikola Vucevic
0.17611764705882352
----------
21801015
Nikola Vucevic
0.3277058823529412
----------
21800996
Nikola Vucevic
0.26341176470588235
--

21800691
Russell Westbrook
0.2331176470588235
----------
21800670
Russell Westbrook
0.2645294117647059
----------
21800653
Russell Westbrook
0.29658823529411765
----------
21800634
Russell Westbrook
0.3089411764705883
----------
21800619
Russell Westbrook
0.23294117647058823
----------
21800603
Russell Westbrook
0.23400000000000004
----------
21800588
Russell Westbrook
0.36811764705882355
----------
21800575
Russell Westbrook
0.2376470588235294
----------
21800562
Russell Westbrook
0.33317647058823524
----------
21800547
Russell Westbrook
0.2647058823529412
----------
21800539
Russell Westbrook
0.21158823529411766
----------
21800525
Russell Westbrook
0.3295882352941176
----------
21800498
Russell Westbrook
0.2998823529411765
----------
21800493
Russell Westbrook
0.2515294117647059
----------
21800485
Russell Westbrook
0.22652941176470587
----------
21800466
Russell Westbrook
0.31994117647058823
----------
21800447
Russell Westbrook
0.255235294117647
----------
21800434
Russell Westbro

21800902
D.J. Wilson
0.18305882352941177
----------
21800854
D.J. Wilson
0.09247058823529412
----------
21800843
D.J. Wilson
0.082
----------
21800833
D.J. Wilson
0.10888235294117647
----------
21800821
D.J. Wilson
0.2421764705882353
----------
21800806
D.J. Wilson
0.1598823529411765
----------
21800792
D.J. Wilson
0.19352941176470587
----------
21800778
D.J. Wilson
0.11264705882352942
----------
21800766
D.J. Wilson
0.2945294117647059
----------
21800750
D.J. Wilson
0.1547058823529412
----------
21800738
D.J. Wilson
0.16041176470588236
----------
21800727
D.J. Wilson
0.08294117647058824
----------
21800694
D.J. Wilson
0.05594117647058824
----------
21800682
D.J. Wilson
0.2997058823529412
----------
21800660
D.J. Wilson
0.14470588235294118
----------
21800654
D.J. Wilson
0.20894117647058824
----------
21800640
D.J. Wilson
0.252764705882353
----------
21800622
D.J. Wilson
0.14370588235294118
----------
21800580
D.J. Wilson
0.12170588235294118
----------
21800572
D.J. Wilson
0.1047058823

21800719
Ivica Zubac
0.2542352941176471
----------
21800701
Ivica Zubac
0.26117647058823534
----------
21800686
Ivica Zubac
0.3225294117647059
----------
21800670
Ivica Zubac
0.20929411764705885
----------
21800656
Ivica Zubac
0.19776470588235295
----------
21800644
Ivica Zubac
0.16558823529411765
----------
21800627
Ivica Zubac
0.2898235294117647
----------
21800616
Ivica Zubac
0.21152941176470588
----------
21800596
Ivica Zubac
0.3972352941176471
----------
21800585
Ivica Zubac
0.30082352941176466
----------
21800574
Ivica Zubac
0.31676470588235295
----------
21800562
Ivica Zubac
0.2058823529411765
----------
21800526
Ivica Zubac
0.30811764705882355
----------
21800514
Ivica Zubac
0.23235294117647062
----------
21800500
Ivica Zubac
0.20105882352941176
----------
21800496
Ivica Zubac
0.2651176470588235
----------
21800478
Ivica Zubac
0.20905882352941174
----------
21800453
Ivica Zubac
0.1323529411764706
----------
21800439
Ivica Zubac
0.07917647058823529
----------
21800430
Ivica Zuba

In [22]:
tp_list.sort(reverse=True)

In [23]:
tp_list

[0.574764705882353,
 0.5398823529411765,
 0.5269411764705882,
 0.5201764705882352,
 0.5102941176470589,
 0.5064117647058823,
 0.5050588235294118,
 0.4999411764705883,
 0.49076470588235294,
 0.4845294117647059,
 0.47058823529411764,
 0.47058823529411764,
 0.46929411764705886,
 0.4638235294117647,
 0.4636470588235294,
 0.4565882352941177,
 0.4518823529411765,
 0.45176470588235296,
 0.44882352941176473,
 0.44817647058823534,
 0.4472941176470589,
 0.4425294117647059,
 0.4419411764705882,
 0.44147058823529417,
 0.4401176470588235,
 0.43870588235294117,
 0.4384705882352941,
 0.4381176470588235,
 0.43658823529411767,
 0.434764705882353,
 0.4341764705882353,
 0.433235294117647,
 0.43241176470588233,
 0.4323529411764706,
 0.43129411764705883,
 0.431,
 0.43070588235294116,
 0.43064705882352944,
 0.42976470588235294,
 0.42882352941176477,
 0.42847058823529416,
 0.42788235294117644,
 0.42776470588235294,
 0.42717647058823527,
 0.4258823529411765,
 0.4255882352941177,
 0.4252941176470588,
 0.424294

In [33]:
usages = df['USG_PCT_x']
usages.sort_values()

6231     0.000
16676    0.000
6643     0.000
16671    0.000
20137    0.000
6642     0.000
6640     0.000
6635     0.000
6634     0.000
11566    0.000
16665    0.000
6633     0.000
6632     0.000
6631     0.000
16647    0.000
20170    0.000
6623     0.000
20185    0.000
16643    0.000
6612     0.000
6608     0.000
6604     0.000
16640    0.000
6598     0.000
3633     0.000
20199    0.000
3641     0.000
16633    0.000
16631    0.000
11634    0.000
         ...  
15827    0.500
18233    0.500
6626     0.500
1829     0.500
6636     0.500
17546    0.500
17388    0.500
6620     0.500
20279    0.516
8909     0.531
15688    0.538
8884     0.543
22400    0.545
8918     0.550
251      0.571
5770     0.571
8913     0.577
21344    0.600
14738    0.600
19544    0.625
18338    0.667
9965     0.667
16615    0.667
3292     0.667
7291     0.667
15680    0.667
11635    0.750
16578    0.833
3628     1.000
5055     1.000
Name: USG_PCT_x, Length: 22462, dtype: float64